In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:1

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Bias-of-Vine_Reviews").getOrCreate()

In [3]:
# Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [4]:
# Recreate the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [5]:
# Filter data to retrieve all rows where total_votes >= 20
total_votes_20_df = vine_df.filter(vine_df["total_votes"] >= 20)
total_votes_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|     

In [6]:
# Filter data to retrieve all rows where helpful_votes/total_votes >= 50%
helpful_votes_50_df = total_votes_20_df.filter(total_votes_20_df["helpful_votes"]/total_votes_20_df["total_votes"] >= .5)
helpful_votes_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [7]:
# Filter to retrieve all rows where a review was written as part of Vine Program
vine_paid_df = helpful_votes_50_df.filter(helpful_votes_50_df["vine"] == 'Y')
vine_paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [8]:
# filter to retrieve all rows where a review was not written as part of Vine Program
vine_unpaid_df = helpful_votes_50_df.filter(helpful_votes_50_df["vine"] == 'N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [9]:
# Showing there are no reviews with Vine that fit our previous criteria
vine_count = vine_paid_df.count()
vine_count

0

In [10]:
# Determine number of reviews for vine unpaid
vine_unpaid_count = vine_unpaid_df.count()
vine_unpaid_count

1685

In [11]:
# Determine the number of 5-star reviews for vine unpaid
five_star_reviews = vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == 5).count()
five_star_reviews

631

In [12]:
#Determine the percentage of 5 star reviews for vine unpaid
percentage = five_star_reviews/vine_unpaid_count
percentage

0.3744807121661721